## Construction du tableau de bord de bilan

### Contexte de la donnée: 
<p>Jeu de donnée de l'INSEE<br>
Description ODD et indicatuers <br>
Suivi dans le temps <br>
    <strong><i>[à compléter]</i></strong>
<br></p>

### Objectifs de représentation: 
<p>Comprendre l'évolution des indicateurs en fonction des données collectées.<br>
Obtenir un tableau de bord interactif permettant de se concentrer sur un ODD et ses indicateurs. <br>
Intégrer un maximum d'informations tout en la conservant lisible. <br>
</p>

### Marche à suivre: 
<p>Produire un premier fichier à l'aide de formules simples/brutes me permettant de visualiser le produit fini: <br>
1. Identifier les formules nécessaires pour constuire le tableau de bord,<br>
2. Comprendre le fonctionnement et les méthodes Openpyxl,<br>
3. Générer un classeur Excel regroupant la donnée 'propre', les formules, les graphiques et exploitable par des corps de métiers.<br></p>
<p><br>
Produire un second fichier à l'aide de fonction permettant de simplifier le code et accélérer le traitement.<br> 
1. Factoriser les différentes formules nécessaires à la construction du tableau de bord,<br>
2. Factoriser le code de chacun des tableaux/graphes nécessaires,<br>
3. Imbriquer les fonctions créées.<br></p>

## Etape 1 /  Préparation et nettoyage du plan de travail: 

Il faut créer puis peupler un excel qui servira de produit final. Ici je crée et ajoute les données et métadonnées dans le fichier Excel sous forme de tableaux afin que je puisse utiliser le nom des fonctions et non les numéros de colonnes du tableau. Cela rend les formules plus fiables et exploitables. Pour cela, je passe par quelques étapes de nettoyage et prétraitement de la donnée. 

In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import os
from openpyxl import Workbook   
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook 
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill
from openpyxl.chart import BarChart, Reference, series, PieChart
from openpyxl.styles import Font, Border, Side
from openpyxl.styles import Alignment
from openpyxl.chart.label import DataLabelList                                                                                                                                                      
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.worksheet.formula import ArrayFormula
from openpyxl.utils import quote_sheetname
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string
from openpyxl.worksheet.worksheet import Worksheet
from openpyxl.styles import Alignment, PatternFill
from openpyxl.worksheet.datavalidation import DataValidation



### Nettoyage de la donnée 

In [3]:
data = pd.read_csv('../DS_DEVDUR_data.csv', sep = ";")
data.head()

,ODD,INDICATEUR_DEVDUR,UNIT_MEASURE,UNIT_MULT,OBS_STATUS,GEO,GEO_OBJECT,DEVDUR_COMPOSITE,SEX,AGE,EMPSTA,PCS,TYPE_INDICATEUR_DEVDUR,TIME_PERIOD,OBS_VALUE
0,ODD1,1.i1a,PT,0,A,FM,FRANCE,NIVIEMED_S60,_T,_T,11,_T,I_ODD,2015,6.5
1,ODD3,3.i2,_Z,0,E,F,FRANCE,CONTAMIN_MOD4,_T,_T,_Z,_Z,I_ODD,2021,255.0
2,ODD2,2.i2f,PT,0,L,F,FRANCE,_Z,_T,_Z,_Z,1_2,I_ODD,2015,NaN
3,ODD1,1.i1a,PT,0,A,FM,FRANCE,NIVIEMED_S60,_T,Y40T49,_T,_T,I_ODD,2016,13.5
4,ODD11,11.i1,PT,0,L,FM,FRANCE,_Z,_T,_T,_Z,_Z,I_ODD,2015,NaN


In [4]:
meta = pd.read_csv('../DS_DEVDUR_metadata.csv',sep = ";")
meta.head()

,COD_VAR,LIB_VAR,COD_MOD,LIB_MOD
0,AGE,Âge,Y18T75,De 18 à 75 ans
1,AGE,Âge,Y18T64,De 18 à 64 ans
2,AGE,Âge,_Z,Non applicable
3,AGE,Âge,Y15,15 ans
4,AGE,Âge,Y_LT14,Moins de 14 ans


Je veux maintenant associer les libellé des codes attribués aux modalités de mon jeu de données. Cela permettra d'avoir une base propre, lisible et directement exploitable. De plus, l'utilisateur pourra retrouver l'information qu'il cherche plus facilement dans la base en cas de doute.<br>

On opère plusieurs fitlre: <br>
1- Données à partir de 2010, date à laquelle on commence à suivre la majorité des indicateurs. <br>
=> limiter la plage temporelle d'analyse, <br>
=> faciliter la lecture (pas de données incohérentes ou de fausses conjonctures sur des données inexistantes).<br>

2- Données avec des valeurs (OBS_STATUS = "L").<br>
<i>OBS_STATUS décrit l'état de l'indicateur. On cherche donc à retirer les indicateurs pour lesquels la valeur est dite 'manquante', encodé 'L'.</i><br>
=> limiter le nombre de lignes de la base pour accélérer le traitement,<br>
=> raccourcir les formules en évitant de filter les countifs sur ce critère par exemple.<br>


In [5]:
## Création d'une table lisible en associant les libellés aux données encodées: 

data_lib = data.copy()
data_lib = data_lib[(data_lib['TIME_PERIOD']>=2010) & (data_lib['OBS_STATUS']!="L")]
cols = ['ODD', 'INDICATEUR_DEVDUR', 'UNIT_MEASURE', 'OBS_STATUS', 'GEO', 'GEO_OBJECT','DEVDUR_COMPOSITE', 'SEX', 'AGE', 'EMPSTA', 'PCS', 'TYPE_INDICATEUR_DEVDUR']
for col in cols:
    if col in meta['COD_VAR'].unique():
        sous_table = meta[meta['COD_VAR'] == col]
        dict_col = dict(zip(sous_table['COD_MOD'],sous_table['LIB_MOD']))
        data_lib[col] = data_lib[col].map(dict_col)
        variables = list(meta['LIB_VAR'].unique())

        # TODO: manque colonne GEO_OBJECT et OBS_VALUE:
dict_var = dict(zip(meta['COD_VAR'],meta['LIB_VAR']))
data_lib.rename(columns=dict_var, inplace=True)


Afin de faciliter le traitement de la donnée dans les formules, on applique une bonne pratique Excel: l'utilisation de tableaux. Au lieu de charger les bases telles qu'elles, on passe par une étape de transformation qui permet de les charger sous forme de tableau en utilisant la méthode <i><strong>Table(displayName=table_name, ref=table_range)</i></strong>. <br>

De plus, cela permettra aux utilisateurs d'intéragir plus fcilement avec la base et de filtrer les éléments directement (sans avoir à ajouter les filtres). <br>

Finalement, l'utilisation de tableau simplifie également l'adaptation des dimensions dans les formules. Grâce au tableau, on utilise directement le nom de la colonne. La dimension appliquée sera celle du tableau. 


In [6]:
## Initialisation classeur output
path_file = '../OUTPUT/TDB_ODD.xlsx'

if not os.path.exists(path_file):
    wb = Workbook()
    wb.save(path_file)

with pd.ExcelWriter(path_file, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    data_lib.to_excel(writer, sheet_name='Base propre', index=False)
    data.to_excel(writer, sheet_name = 'Base brute', index = False)
    meta.to_excel(writer, sheet_name='Index', index=False)

from openpyxl.worksheet.table import Table, TableStyleInfo

wb = load_workbook(path_file)

def add_table(ws, df, table_name):
    n_rows, n_cols = df.shape
    max_row = n_rows + 1  
    max_col = n_cols
    
    last_col_letter = get_column_letter(max_col)
    
    table_range = f"A1:{last_col_letter}{max_row}"  
    
    tab = Table(displayName=table_name, ref=table_range)
    style = TableStyleInfo(name="TableStyleMedium3", showRowStripes=True)
    tab.tableStyleInfo = style
    
    ws.add_table(tab)

# Ajouter tableaux dans les feuilles concernées
add_table(wb['Base propre'], data_lib, "Base")
add_table(wb['Index'], meta, "Index")

wb.save(path_file)

## Deuxième étape // Préparation du document de sortie

On initialise les feuilles de résultats.<br>
On conditionne ensuite l'affichage sur deux filtres.

In [7]:
## Initialisation feuille bilan 

wb = load_workbook(path_file)
if 'TDB' not in wb.sheetnames:
    wb.create_sheet('TDB')
    wb.save(path_file)
    print("Feuille bilan créée.")
else:
    print("Feuille bilan trouvée.")
    

Feuille bilan créée.


Avant toute chose, on ajoute les filtres qui définissent le reste des affichages. On cherche à avoir deux filtres: année + objetif de développement durable. <br>
Ces filtres utilisent la méthode DataValidation propre à Excel qui permet d'établir des listes de valeurs stockées dans une cellule. <br>

<strong>L'objectif de développement durable</strong>: Cela permettra de filtrer le reste des affichages en fonction des observations liées à cet indicateur.<br> 
<strong>L'année</strong>: cela permet de délimiter un peu plus le périmètre d'analyse et d'identifier des évolutions dans le temps. <i>Tous les affichages ne seront pas limités à ce filtre ci</i>.<br>


In [8]:
## Filtre ODD

bilan = wb['TDB']

titre_f1 = bilan['A1']
titre_f1.value = 'ODD'
titre_f1.fill = PatternFill(start_color='8DB600', end_color='8DB600', fill_type='solid')

bilan.merge_cells(start_row=1, 
                       start_column=1, 
                       end_row=2, 
                       end_column=2)

val_F1 = bilan['C1']
val_F1.value = 'Objectif 1 : Éradication de la pauvreté'
val_F1.alignment  = Alignment(horizontal='center', vertical='center')
val_F1.fill = PatternFill(start_color='8DB600', end_color='8DB600', fill_type='solid')

formula = "='Base propre'!A:A"

dv = DataValidation(type='list', formula1=formula)
bilan.add_data_validation(dv)
coord_filter_odd = 'C1'
dv.add(coord_filter_odd)

bilan.merge_cells(start_row=1, 
                       start_column=3, 
                       end_row=2, 
                       end_column=6)

wb.save(path_file)
wb.close()

In [9]:
## Filtre années

bilan = wb['TDB']

titre_f2 = bilan['H1']
titre_f2.value = 'Année'
titre_f2.fill = PatternFill(start_color='8DB600', end_color='8DB600', fill_type='solid')

bilan.merge_cells(start_row=1, 
                       start_column=8, 
                       end_row=2, 
                       end_column=9)

val_F2 = bilan['J1']
val_F2.value = '2022'
val_F2.alignment  = Alignment(horizontal='center', vertical='center')
val_F2.fill = PatternFill(start_color='8DB600', end_color='8DB600', fill_type='solid')

formula = "='Base propre'!N:N"

dv = DataValidation(type='list', formula1=formula)
bilan.add_data_validation(dv)
coord_filter_year = 'J1'
dv.add(coord_filter_year)

bilan.merge_cells(start_row=1, 
                       start_column=10, 
                       end_row=2, 
                       end_column=15)

wb.save(path_file)
wb.close()

## Etape 3 // Initialisation des indicateurs 

On ajoute à chaque fois un tableau et un visuel associé qui permet de contextualiser et souligner l'information importante. 

1- Tableau descriptif des indicateurs principaux qui décrivent l'objectif de développement durable choisi sur l'année choisie (filtres). <br>
    Pas de visuel associé. 
    
2- Tableau de volumes d'indicateurs suivi au cours du temps. <br>
    Histogramme en barres.

3- Tableau de répartition des observations renseignées par composite de développement durable. <br>
    Histogramme en barres.

In [10]:
## Tableau description ODD

titre_tab = bilan['B4']
titre_tab.value = "Observations cumulées par indicateur"

titre_tab.fill = PatternFill(start_color='FF8DB600', end_color='FF8DB600', fill_type='solid')

titre_tab.alignment = Alignment(horizontal='center', vertical='center', wrap_text = True)

bilan.merge_cells(start_row=4, start_column=2, end_row=10, end_column=3)

formula = f"=_xlfn.UNIQUE(_xlfn.FILTER(Base[Indicateur d’objectif de développement durable ],(Base[Objectif développement durable]) = $C$1))"

bilan['D4'] = "Indicateurs"
bilan['D5'] = ArrayFormula(f"D5:D10", formula )

bilan['E4'] = "Moyenne des observations"
bilan ['E5'] = f"=_xlfn.IFERROR(AVERAGEIFS(Base[OBS_VALUE], Base[Indicateur d’objectif de développement durable ],D5, Base[Période temporelle],J1),\"-\")"
bilan ['E6'] = f"=_xlfn.IFERROR(AVERAGEIFS(Base[OBS_VALUE],Base[Indicateur d’objectif de développement durable ],D6, Base[Période temporelle],J1),\"-\")"
bilan ['E7'] = f"=_xlfn.IFERROR(AVERAGEIFS(Base[OBS_VALUE],Base[Indicateur d’objectif de développement durable ],D7, Base[Période temporelle],J1),\"-\")"
bilan ['E8'] = f"=_xlfn.IFERROR(AVERAGEIFS(Base[OBS_VALUE],Base[Indicateur d’objectif de développement durable ],D8, Base[Période temporelle],J1),\"-\")"
bilan ['E9'] = f"=_xlfn.IFERROR(AVERAGEIFS(Base[OBS_VALUE],Base[Indicateur d’objectif de développement durable ],D9, Base[Période temporelle],J1),\"-\")"
bilan ['E10'] = f"=_xlfn.IFERROR(AVERAGEIFS(Base[OBS_VALUE],Base[Indicateur d’objectif de développement durable ],D10, Base[Période temporelle],J1),\"-\")"

bilan['F4'] = "Unité de mesure" # Ajouter une autre colonne observation que l'on multiplie par unit_mult pour avoir la val totale
bilan ['F5'] = f'=_xlfn.XLOOKUP(D5,Base[Indicateur d’objectif de développement durable ],Base[Unité de mesure],"-")'
bilan ['F6'] = f'=_xlfn.XLOOKUP(D6,Base[Indicateur d’objectif de développement durable ],Base[Unité de mesure],"-")'
bilan ['F7'] = f'=_xlfn.XLOOKUP(D7,Base[Indicateur d’objectif de développement durable ],Base[Unité de mesure],"-")'
bilan ['F8'] = f'=_xlfn.XLOOKUP(D8,Base[Indicateur d’objectif de développement durable ],Base[Unité de mesure],"-")'
bilan ['F9'] = f'=_xlfn.XLOOKUP(D9,Base[Indicateur d’objectif de développement durable ],Base[Unité de mesure],"-")'
bilan ['F10'] = f'=_xlfn.XLOOKUP(D10,Base[Indicateur d’objectif de développement durable ],Base[Unité de mesure],"-")'

wb.save(path_file)
wb.close()

In [11]:
## Données années

titre_tab2 = bilan['B20']
titre_tab2.value = "Observations cumulées par année"

titre_tab2.fill = PatternFill(start_color='FF7F00', end_color='FF7F00', fill_type='solid')

titre_tab2.alignment = Alignment(horizontal='center', vertical='center', wrap_text = True)

bilan.merge_cells(start_row=20, start_column=2, end_row=33, end_column=3)

formula = f"=_xlfn.SORT(_xlfn.UNIQUE(_xlfn.FILTER(Base[Période temporelle],(Base[Objectif développement durable]) = $C$1)))"

bilan['D20'] = "Années"
bilan['D21'] = ArrayFormula(f"D21:D33", formula )

bilan['E20'] = "Nbr d'observations"
for row in range(21, 34):  # de D21 à D33 → 13 lignes
    cell_d = f"D{row}"
    cell_e = f"E{row}"
    bilan[cell_e] = f'=COUNTIFS(Base[Période temporelle], {cell_d}, Base[Objectif développement durable], $C$1)'


wb.save(path_file)
wb.close()

In [12]:
## Graph années
chart = BarChart()
chart.title = "Evolution du nombre d'indicateurs suivis par an"
chart.x_axis.title = "Année"
chart.y_axis.title = "Indicateurs"

values = Reference(bilan, min_col=5, min_row=20, max_col = 5, max_row=33)
axes = Reference(bilan, min_col=4, min_row=20, max_col = 5, max_row=20)

chart.add_data(values, titles_from_data=True)
chart.set_categories(axes)

bilan.add_chart(chart, "I22")

wb.save(path_file)


In [13]:
## Données composites

titre_tab3 = bilan['B40']
titre_tab3.value = "Répartition des indicateurs par composite"
titre_tab3.fill = PatternFill(start_color='FD6C9E', end_color='FD6C9E', fill_type='solid')
titre_tab3.alignment = Alignment(horizontal='center', vertical='center', wrap_text = True)
bilan.merge_cells(start_row=40, start_column=2, end_row=50, end_column=3)

formula = f"=_xlfn.IFERROR(_xlfn.SORT(_xlfn.UNIQUE(_xlfn.FILTER(Base[Composite développement durable],(Base[Objectif développement durable]) = $C$1))),\"-\")"

bilan['D39'] = "Composite développement durable"
bilan['D41'] = ArrayFormula(f"D40:D50", formula )

bilan['E39'] = "Nbr d'observations"
bilan ['G39'] = "Nbr de composites de l'ODD"
bilan ['G40'] = f"=COUNTA(_xlfn.UNIQUE(_xlfn.FILTER(Base[Composite développement durable], Base[Objectif développement durable] = TDB!C1)))"

for row in range(40, 50):  
    cell_d = f"D{row}"
    cell_e = f"E{row}"
    #cell_f = f"F{row}"

    bilan[cell_e] = (
        f'=IF({cell_d}="", "", '
        f'_xlfn.IFERROR(COUNTIFS(Base[Composite développement durable], {cell_d}, '
        f'Base[Objectif développement durable], $C$1), "-"))'
    )
    #bilan[cell_f] = (
     #   f'=F{row-1} + 1'
      #  )

wb.save(path_file)
wb.close()

In [14]:
## Graphique composite
chart2 = BarChart()
chart2.title = "Répartition des indicateurs suivis par composite de développement durable"
chart2.x_axis.title = "Composite"
chart2.y_axis.title = "Indicateurs"

values_2 = Reference(bilan, min_col=5, min_row=39, max_col = 5, max_row=50)
axes_2 = Reference(bilan, min_col=4, min_row=40, max_col = 4, max_row=49)

chart2.add_data(values_2, titles_from_data=True)
chart2.set_categories(axes_2)

bilan.add_chart(chart2, "I42")

wb.save(path_file)


<strong>A ajouter</strong>: un pie chart au lieu de l'histogramme en 3. Un treemap des ODD en général qui permette de visualiser la place qu'ils prennent. <br>
Rediriger l'ouverture du fichier sur la page "TDB".<br>
Un encadré sur les populations ciblées:<br> 
pie chart sexe. <br>
pie chart des ages.<br>
pie chart statut emploi. <br>
\+ une phrase adaptable sur le filtre de l'année et deux count (la population, lage et l'emploi le plus concerné et par ODD si les filtres sont séléctionnés).
